In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.options.display.max_info_columns=500
pd.options.display.max_info_rows=550
from tensorflow.keras.losses import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn import set_config; set_config(display='diagram')
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score, GridSearchCV
from xgboost import XGBRegressor

2022-09-09 14:38:18.015866: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-09 14:38:18.015903: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
from carbon_forecast.utils.preproc import load_n_preproc
df2013 = load_n_preproc(2013)
df2014 = load_n_preproc(2014)
df2015 = load_n_preproc(2015)
df2016 = load_n_preproc(2016)
df2017 = load_n_preproc(2017)
df2018 = load_n_preproc(2018)
df2019 = load_n_preproc(2019)
df2020 = load_n_preproc(2020)


Reading data...
Preproc on power % revenue
Preproc on Sector Revenue
Selecting the year 2013 required...
Dropping rows that are showing NaN on...
-> intensity_scope1CY2013
-> intensity_scope2CY2013
-> intensity_scope3CY2013
-> Revenue2013
Filling employee numbers with mean value... (subject to change later)
Filling EV with mean value... (subject to change later)
Calculating Employees & EV per Revenue...
Filling null Climate Strategy Score with 0...
Cleaning P/E Ratio...
Adding Scope 1 and Scope 2 data together...
Doing some column renaming...
Dataset preproc-ed for the year 2013!👍
Reading data...
Preproc on power % revenue
Preproc on Sector Revenue
Selecting the year 2014 required...
Dropping rows that are showing NaN on...
-> intensity_scope1CY2014
-> intensity_scope2CY2014
-> intensity_scope3CY2014
-> Revenue2014
Filling employee numbers with mean value... (subject to change later)
Filling EV with mean value... (subject to change later)
Calculating Employees & EV per Revenue...
Filli

In [4]:
df2013.shape, df2014.shape ,df2015.shape,df2016.shape,df2017.shape,df2018.shape,df2019.shape,df2020.shape

((4672, 463),
 (5054, 463),
 (5254, 463),
 (11863, 463),
 (12943, 463),
 (13889, 463),
 (14785, 463),
 (16639, 463))

In [5]:
big = pd.concat([df2013, df2014, df2015, df2016, df2017, df2018, df2019, df2020])
big

,company_name,TCUID,year,Sector,Employees / Revenue,EV / Revenue,climate_strategy_score,disclosure,intensity_1and2,intensity_scope3,pe_rat,revenue,nuclear_percentage_revenue,thermal_coal_percentage_revenue,ultra_deep_sea_drilling_percentage_revenue,shale_oil_gas_percentage_revenue,coal_percentage_revenue,arctic_drilling_percentage_revenue,oil_sands_percentage_revenue,Abrasive product manufacturing,"Accounting, tax preparation, bookkeeping, and payroll services",Adhesive manufacturing,Advertising and related services,Air and gas compressor manufacturing,"Air conditioning, refrigeration, and warm air heating equipment manufacturing",Air purification and ventilation equipment manufacturing,Air transportation,Aircraft engine and engine parts manufacturing,Aircraft manufacturing,Alkalies and chlorine manufacturing,All other basic inorganic chemical manufacturing,All other chemical product and preparation manufacturing,All other converted paper product manufacturing,All other crop farming,All other food manufacturing,"All other forging, stamping, and sintering",All other miscellaneous electrical equipment and component manufacturing,All other miscellaneous manufacturing,"All other miscellaneous professional, scientific, and technical services",All other miscellaneous wood product manufacturing,All other paper bag and coated and treated paper manufacturing,All other petroleum and coal products manufacturing,All other textile product mills,All other transportation equipment manufacturing,Alumina refining and primary aluminum production,Aluminum product manufacturing from purchased aluminum,Ammunition manufacturing,"Amusement parks, arcades, and gambling industries",Analytical laboratory instrument manufacturing,"Animal (except poultry) slaughtering, rendering, and processing","Animal production, except cattle and poultry and eggs",Apparel accessories and other apparel manufacturing,Apparel knitting mills,"Apparel, Piece Goods, and Notions Wholesalers","Architectural, engineering, and related services","Arms, ordnance, and accessories manufacturing",Artificial and synthetic fibers and filaments manufacturing,Asphalt paving mixture and block manufacturing,Asphalt shingle and coating materials manufacturing,Audio and video equipment manufacturing,Automatic environmental control manufacturing,Automobile manufacturing,Automotive equipment rental and leasing,"Automotive repair and maintenance, except car washes",Ball and roller bearing manufacturing,Bare printed circuit board manufacturing,Bauxite Mining,Beet sugar manufacturing,Biological product (except diagnostic) manufacturing,Biomass Power Generation,Bituminous Coal Underground Mining,Bituminous Coal and Lignite Surface Mining,Blind and shade manufacturing,Boat building,Book publishers,Bowling centers,Bread and bakery product manufacturing,Breakfast cereal manufacturing,Breweries,"Brick, tile, and other structural clay product manufacturing",Broadcast and wireless communications equipment,Broadwoven fabric mills,"Broom, brush, and mop manufacturing",Building Material and Garden Equipment and Supplies Dealers,Business support services,Cable and other subscription programming,Car washes,Carbon and graphite product manufacturing,Carbon black manufacturing,Carpet and rug mills,Cattle ranching and farming,Cement manufacturing,Cheese manufacturing,Child day care services,Chocolate and confectionery manufacturing from cacao beans,Clay and nonclay refractory manufacturing,Clothing and Clothing Accessories Stores,Coal Power Generation,"Coated and laminated paper, packaging paper and plastics film manufacturing","Coating, engraving, heat treating and allied activities",Coffee and tea manufacturing,Commercial and industrial machinery and equipment rental and leasing,Commercial and industrial machinery and equipment repair and maintenance,Communication and energy wire and cable manufacturing,"Community food, housing, and other relief services, including rehabilitation services",Computer storage de

In [6]:
X = big.drop(columns=["company_name", "TCUID", "year", "intensity_1and2","intensity_scope3"])
y = big["intensity_1and2"]

In [7]:
y_baseline = y.mean()
y_baseline

344.9996143848601

In [8]:
mean_absolute_error(y,y_baseline)

2022-09-09 14:38:26.258057: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-09 14:38:26.258100: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-09 14:38:26.258119: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (VIVOLC): /proc/driver/nvidia/version does not exist
2022-09-09 14:38:26.258974: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(), dtype=float64, numpy=498.3965359160297>

In [9]:
num_col = sorted(X.select_dtypes(include=["int64", "float64"]).columns)
cat_col = sorted(list(set(X.columns) - set(num_col)))
cat_col

['Sector']

In [10]:
num_transformer = MinMaxScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer([
    ('num_tr', num_transformer, num_col),
    ('cat_tr', cat_transformer, cat_col)
],
sparse_threshold=0)

In [11]:
preprocessor

ColumnTransformer(sparse_threshold=0,
                  transformers=[('num_tr', MinMaxScaler(),
                                 ['Abrasive product manufacturing',
                                  'Accounting, tax preparation, bookkeeping, '
                                  'and payroll services',
                                  'Adhesive manufacturing',
                                  'Advertising and related services',
                                  'Air and gas compressor manufacturing',
                                  'Air conditioning, refrigeration, and warm '
                                  'air heating equipment manufacturing',
                                  'Air purification...
                                  'All other textile product mills',
                                  'All other transportation equipment '
                                  'manufacturing',
                                  'Alumina refining and primary aluminum '
                                  'production',
                                  'Aluminum product manufacturing from '
                                  'purchased aluminum',
                                  'Ammunition manufacturing',
                                  'Amusement parks, arcades, and gambling '
                                  'industries',
                                  'Analytical laboratory instrument '
                                  'manufacturing', ...]),
                                ('cat_tr',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Sector'])])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [14]:
model = XGBRegressor(n_estimators=1000, max_depth=5, objective="reg:squarederror", eval_metric="mape")
cross_val_score(model, X_train_transformed, y_train, cv=5, scoring="neg_mean_absolute_percentage_error")

array([ -6.18496519, -11.8275928 ,  -1.47568674,  -4.46089638,
        -5.8260968 ])

In [15]:
from carbon_forecast.utils.preproc import load_n_preproc, load_n_preproc_all_avail
dd = load_n_preproc_all_avail()

Reading data...
Preproc on power % revenue
Preproc on Sector Revenue
Selecting the year 2013 required...
Dropping rows that are showing NaN on...
-> intensity_scope1CY2013
-> intensity_scope2CY2013
-> intensity_scope3CY2013
-> Revenue2013
Filling employee numbers with mean value... (subject to change later)
Filling EV with mean value... (subject to change later)
Calculating Employees & EV per Revenue...
Filling null Climate Strategy Score with 0...
Cleaning P/E Ratio...
Adding Scope 1 and Scope 2 data together...
Doing some column renaming...
Dataset preproc-ed for the year 2013!👍
Reading data...
Preproc on power % revenue
Preproc on Sector Revenue
Selecting the year 2014 required...
Dropping rows that are showing NaN on...
-> intensity_scope1CY2014
-> intensity_scope2CY2014
-> intensity_scope3CY2014
-> Revenue2014
Filling employee numbers with mean value... (subject to change later)
Filling EV with mean value... (subject to change later)
Calculating Employees & EV per Revenue...
Filli

In [28]:
model2 = XGBRegressor(n_estimators=1000, max_depth=10, objective="reg:squarederror", eval_metric="mape")
cross_val_score(model2, X_train_transformed, y_train, cv=5, scoring="neg_mean_absolute_percentage_error")

array([-2.43791275, -1.02354273, -0.96207181, -3.49784379, -4.76646527])